In [1]:
import os
from unittest import result
from Bio.PDB import PDBParser
from Bio.PDB.NeighborSearch import NeighborSearch
import glob
import multiprocessing

# def is_illegal_pdb(pdb_file, chain_a_id='A', chain_b_id='B'):
#     parser = PDBParser(QUIET=True)
#     structure = parser.get_structure('structure', pdb_file)

#     # Extract non-hydrogen atoms from chain A and chain B
#     chain_a_atoms = [atom for atom in structure.get_atoms() if atom.get_parent().get_parent().id == chain_a_id and atom.element != 'H']
#     chain_b_atoms = [atom for atom in structure.get_atoms() if atom.get_parent().get_parent().id == chain_b_id and atom.element != 'H']
    
#     # Check for clashes between chain A and chain B atoms
#     neighbor_search = NeighborSearch(chain_a_atoms)
    
#     for atom in chain_b_atoms:
#         neighbors = neighbor_search.search(atom.coord, 2.1)
#         if neighbors:
#             return True
    
#     return False

def nearest_distance(pdb_file, chain_a_id='A', chain_b_id='B'):
    try:
        parser = PDBParser(QUIET=True)
        structure = parser.get_structure('structure', pdb_file)

        # Extract non-hydrogen atoms from chain A and chain B
        chain_a_atoms = [atom for atom in structure.get_atoms() if atom.get_parent().get_parent().id == chain_a_id and atom.element != 'H']
        chain_b_atoms = [atom for atom in structure.get_atoms() if atom.get_parent().get_parent().id == chain_b_id and atom.element != 'H']
        
        min_distance = float('inf')
        
        for atom in chain_b_atoms:
            for neighbor in chain_a_atoms:
                distance = atom - neighbor
                if distance < min_distance:
                    min_distance = distance
        
        # for atom in chain_a_atoms:
        #     for neighbor in chain_b_atoms:
        #         distance = atom - neighbor
        #         if abs(distance - min_distance) < 1e-6:
        #             print(atom.get_parent().get_parent().id, atom.get_name(), neighbor.get_parent().get_parent().id, neighbor.get_name(), distance)

        return min_distance
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except Exception as e:
        print (pdb_file)
        print(f'Error: {e}')
        return None

pdb_dir = '/data/DTWG_AF2_homoaug_complex/'
pdb_files = glob.glob(os.path.join(pdb_dir, '*refined.pdb'))


distance_dist=[]

with multiprocessing.Pool() as pool:
    results=pool.map(nearest_distance, pdb_files)




/data/DTWG_AF2_homoaug_complex/AF-O75369-F1-model_v4_20_5o1f_0_1_complex_refined.pdb
Error: Invalid or missing coordinate(s) at line 1224.
/data/DTWG_AF2_homoaug_complex/AF-Q9NTI2-F1-model_v4_0_4j2t_1_2_complex_refined.pdb
Error: Invalid or missing coordinate(s) at line 4125.
/data/DTWG_AF2_homoaug_complex/AF-Q8NDX2-F1-model_v4_0_4zyr_0_0_complex_refined.pdb
Error: Invalid or missing coordinate(s) at line 368.
/data/DTWG_AF2_homoaug_complex/AF-Q8NGP3-F1-model_v4_0_6rz6_2_1_complex_refined.pdb
Error: Invalid or missing coordinate(s) at line 131.
/data/DTWG_AF2_homoaug_complex/AF-Q96Q04-F1-model_v4_0_1yi3_0_1_complex_refined.pdb
Error: Invalid or missing coordinate(s) at line 224.
/data/DTWG_AF2_homoaug_complex/AF-Q9UHV8-F1-model_v4_0_1lhv_1_4_complex_refined.pdb
Error: Invalid or missing coordinate(s) at line 617.
/data/DTWG_AF2_homoaug_complex/AF-Q9HD43-F1-model_v4_0_2nt7_0_2_complex_refined.pdb
Error: Invalid or missing coordinate(s) at line 525.
/data/DTWG_AF2_homoaug_complex/AF-O152

In [4]:
from tqdm import tqdm
cnt=0
for pdb_file , result in tqdm(zip(pdb_files, results)):
    if result is None or result < 2.5 or result > 3.5:
        # move to /data/DTWG_AF2_homoaug_complex_illegal
        cmd=f'mv {pdb_file} /data/DTWG_AF2_homoaug_complex_illegal/'
        os.system(cmd)
        cnt+=1

print(f'{cnt} illegal pdbs moved')


55855it [00:32, 1703.77it/s]

3880 illegal pdbs moved


In [3]:
import sys
sys.path.append("/project/Drug_The_Whole_Genome/")
import os
import multiprocessing
from Bio.PDB import PDBParser
from utils.Dataset import PDBBindDataset

def calc_num_chains(item):
    # print (pocket_dir)
    # use biopython
    pocket_dir=item['pocket10A_dir']
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('structure', pocket_dir)
    chains = set()
    for model in structure:
        for chain in model:
            if chain.id != ' ':
                chains.add(chain.id)
    # print(chains)
    return len(chains)

dataset=PDBBindDataset()

with multiprocessing.Pool() as pool:
    num_chains=pool.map(calc_num_chains, dataset.get_items())


In [6]:
single_pocket_case=[]

for item, num_chain in zip(dataset.get_items(), num_chains):
    if num_chain==1:
        single_pocket_case.append(item['name'])
print (single_pocket_case)
print(len(single_pocket_case))

# save to /data/pdbbind_2020/single_chain_pocket10A.pkl
import pickle
with open('/data/pdbbind_2020/single_chain_pocket10A.pkl', 'wb') as f:
    pickle.dump(single_pocket_case, f)

['2pjt', '6dji', '2hdr', '3eta', '5oq4', '5ogl', '3oka', '4hg7', '5vb5', '4u0m', '2bz6', '4nrq', '4wx7', '5ywg', '3g90', '1iup', '4ibc', '5jt9', '5evd', '1ggn', '4kqr', '5ifu', '4prj', '4b60', '3px8', '5wcl', '4dce', '5t9w', '5i9z', '3v01', '1yw7', '4jpy', '1xmy', '5tp0', '3gpe', '4bbh', '1x7r', '1lf9', '1vj6', '3kyq', '1q65', '5k32', '3cy2', '1br8', '4k9y', '5zr3', '1y20', '6qpl', '5b6g', '5ghv', '3uqf', '2zu4', '5fqr', '1nw7', '5ygi', '1n1g', '4qvx', '5agi', '4yxi', '3udv', '4m8h', '6hni', '5m6h', '3q0z', '1zgi', '1lkk', '3rk9', '5zbz', '5a5d', '3juk', '1bzh', '1t08', '4zx4', '4j82', '3fhe', '3zm6', '1hk3', '2pzy', '3e6y', '2pe2', '2d1o', '2e9n', '6bbu', '5zjz', '4i6q', '3ccc', '2xm8', '3whw', '4k5l', '4m2w', '4xqu', '2j7g', '5oei', '6ezi', '6gwr', '3kfa', '2vew', '3wd2', '1onh', '2pj3', '4xg3', '4anv', '2v11', '4je8', '4pp7', '1g53', '4fri', '4odl', '8cpa', '4tte', '4nld', '3rqg', '5mk3', '6n7z', '3cgy', '2jt5', '5ti6', '6kzc', '4piq', '6axj', '4tpp', '5khi', '1thl', '4q19', '5ula',

In [27]:

cnt=0
all_complex=glob.glob('/data/DTWG_AF2_homoaug_complex/*.pdb')
print("number of all complex", len(all_complex))
for item in tqdm(all_complex):
    name=item.split('/')[-1].split('_')[3]

    if not name in single_pocket_case:
        cmd=f'mv {item} /data/DTWG_AF2_homoaug_complex_illegal/'
        os.system(cmd)
        cnt+=1
print(f'{cnt} illegal pdbs moved')

['2pjt', '6dji', '2hdr', '3eta', '5oq4', '5ogl', '3oka', '4hg7', '5vb5', '4u0m', '2bz6', '4nrq', '4wx7', '5ywg', '3g90', '1iup', '4ibc', '5jt9', '5evd', '1ggn', '4kqr', '5ifu', '4prj', '4b60', '3px8', '5wcl', '4dce', '5t9w', '5i9z', '3v01', '1yw7', '4jpy', '1xmy', '5tp0', '3gpe', '4bbh', '1x7r', '1lf9', '1vj6', '3kyq', '1q65', '5k32', '3cy2', '1br8', '4k9y', '5zr3', '1y20', '6qpl', '5b6g', '5ghv', '3uqf', '2zu4', '5fqr', '1nw7', '5ygi', '1n1g', '4qvx', '5agi', '4yxi', '3udv', '4m8h', '6hni', '5m6h', '3q0z', '1zgi', '1lkk', '3rk9', '5zbz', '5a5d', '3juk', '1bzh', '1t08', '4zx4', '4j82', '3fhe', '3zm6', '1hk3', '2pzy', '3e6y', '2pe2', '2d1o', '2e9n', '6bbu', '5zjz', '4i6q', '3ccc', '2xm8', '3whw', '4k5l', '4m2w', '4xqu', '2j7g', '5oei', '6ezi', '6gwr', '3kfa', '2vew', '3wd2', '1onh', '2pj3', '4xg3', '4anv', '2v11', '4je8', '4pp7', '1g53', '4fri', '4odl', '8cpa', '4tte', '4nld', '3rqg', '5mk3', '6n7z', '3cgy', '2jt5', '5ti6', '6kzc', '4piq', '6axj', '4tpp', '5khi', '1thl', '4q19', '5ula',

100%|██████████| 157245/157245 [07:15<00:00, 360.70it/s]

41850 illegal pdbs moved
